In [22]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import rotate
from registration_utilities import *

from cell_utilities import *

In [23]:
# dataset_name = 'largest30p'
dataset_name = 'largest30p_sizeNormalized'

In [24]:
CELL_EMBEDDING_DIR = os.path.join(CELL_EMBEDDING_ROOTDIR, dataset_name)

In [25]:
kmeanspp_center_indices = bp.unpack_ndarray_file(os.path.join(CELL_EMBEDDING_DIR, \
                        '%(dataset_name)s_kmeansppCenters.bp' % {'dataset_name': dataset_name}))

closest_center_indices = bp.unpack_ndarray_file(os.path.join(CELL_EMBEDDING_DIR, \
                '%(dataset_name)s_closestKmeansppCenters.bp' % {'dataset_name': dataset_name}))

In [26]:
kmeanspp_center_member_counts = np.bincount(closest_center_indices, minlength=len(kmeanspp_center_indices))

In [28]:
dataset_cells_aligned_padded = bp.unpack_ndarray_file(os.path.join(CELL_EMBEDDING_DIR, '%(dataset_name)s_cellsAlignedPadded.bp' % dict(dataset_name=dataset_name)))

In [29]:
n_dataset_cells = len(dataset_cells_aligned_padded)

In [30]:
D3JS_ROOTDIR = os.path.join(CELLS_ROOTDIR, 'd3js')

In [31]:
# d3js data folder
output_dir = create_if_not_exists(os.path.join(D3JS_ROOTDIR, dataset_name))
! rm -r {output_dir}/*

In [32]:
for nystrom_sample in [10, 100, 300, 1000]:
# for nystrom_sample in [100]:
    
    E = bp.unpack_ndarray_file(os.path.join(CELL_EMBEDDING_DIR, \
                        'embeddings_normalized_%(dataset_name)s_nystromSample%(nystrom_sample)d.bp' % \
                        {'dataset_name': dataset_name, 'nystrom_sample': nystrom_sample}))
    
    for n in [10, 100, 1000]:

        # n = 1000
        colors = random_colors(n)
        indices = np.random.choice(np.arange(n_dataset_cells), n, replace=False)

        # n = 1000
        # colors = random_colors(n)
        # indices = kmeanspp_center_indices

        # n = 10
#         colors = random_colors(n)
#         indices = kmeanspp_center_indices[:n]

        sizes = np.zeros((n, 2), np.int)

        for r, i in enumerate(indices):

            contour_mask = np.zeros_like(dataset_cells_aligned_padded[i])
            contour_pts = np.array(find_contour_points(dataset_cells_aligned_padded[i].astype(np.int), sample_every=1)[1][0], np.int)
            contour_mask[contour_pts[:,1], contour_pts[:,0]] = 1
            xmin, xmax, ymin, ymax = bbox_2d(contour_mask)      
            m = contour_mask[ymin:ymax+1, xmin:xmax+1].copy()

            m_rgba = np.zeros(m.shape+(4,), np.uint8)
            m_rgba[m > 0, :3] = colors[r]
            m_rgba[m > 0, 3] = 255

            contour_output_dir = create_if_not_exists(os.path.join(output_dir, 'random%d/contours' % (n)))
            imsave(os.path.join(contour_output_dir, 'cellMask_random%dContours_%d_%d.png' % (n,r, i)), m_rgba)

            xmin, xmax, ymin, ymax = bbox_2d(dataset_cells_aligned_padded[i])  
            m = dataset_cells_aligned_padded[i][ymin:ymax+1, xmin:xmax+1].copy()

            m_rgba = np.zeros(m.shape+(4,), np.uint8)
            m_rgba[m > 0, :3] = colors[r]
            m_rgba[m > 0, 3] = 255

            blob_output_dir = create_if_not_exists(os.path.join(output_dir, 'random%d/blobs' % (n)))
            imsave(os.path.join(blob_output_dir, 'cellMask_random%dBlobs_%d_%d.png' % (n, r, i)), m_rgba)

            sizes[r] = m_rgba.shape[:2]


        E_dict = []

        for r, i in enumerate(indices):
            e1, e2, e3, e4, e5 = E[i, :5]
            s = sizes[r]
#             E_dict.append({'e1':e1, 'e2':e2, 'e3':e3, 'e4':e4, 'e5':e5, 'id':r, 'index':i, 'w':s[1], 'h':s[0], 
#                            'memberCount': kmeanspp_center_member_counts[r]})
            E_dict.append({'e1':e1, 'e2':e2, 'e3':e3, 'e4':e4, 'e5':e5, 'id':r, 'index':i, 'w':s[1], 'h':s[0], 
                           'memberCount': 1})

        import json
        json.dump(E_dict, open(os.path.join(output_dir, 'random%d/embedding_random%d_nystrom%d.json' % (n, n, nystrom_sample)), 'w'))

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: /home/yuncong/csd395/CSHL_cells_v2/d3js/largest30p_sizeNormalized/random10/contours/cellMask_random10Contours_7_47550.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: /home/yuncong/csd395/CSHL_cells_v2/d3js/largest30p_sizeNormalized/random10/blobs/cellMask_random10Blobs_7_47550.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: /home/yuncong/csd395/CSHL_cells_v2/d3js/largest30p_sizeNormalized/random100/contours/cellMask_random100Contours_14_18657.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/sk

In [33]:
! cd {CELLS_ROOTDIR}; tar czf d3js.tar.gz d3js